# FFTSmooth with POWGEN data 
<div class="alert alert-block alert-info">
<p><b>Information:</b> 
In this notebook we compare the implementation in <code>scipp</code> of <code>FFTSmooth</code> with the one available in <code>Mantid</code>. The implementation in <code>scipp</code> does not reproduce all the options available in <code>Mantid</code>. For example, the only filter available in <code>scipp</code> is <i>Butterworth</i>. </p> 
    
The input datafile (Vanadium) <a href="http://198.74.56.37/ftp/external-data/MD5/3d543bc6a646e622b3f4542bc3435e7e">PG3_4866_event.nxs</a> is part of <code>Mantid</code>'s test data. The input workspace is in units of <i>tof</i>.  

In <code>Mantid</code> <code>FFTSmooth</code> does not preserve the <code>Eventworkspace</code> status. Therefore in <code>scipp</code> we use an histogrammed dataset as input. For comparison, we also rebin the input workspace in <code>Mantid</code>.  
    
<p><b>Requirements:</b>
The Python script <code>fft_smooth.py</code>, which contains the implementation of <code>fft_smooth</code>, has to be in the same directory as this notebook.</p>
    
</div>

## Mantid

We load Vanadium data, select a spectrum, rebin and apply [`FFTSmooth`](https://docs.mantidproject.org/nightly/algorithms/FFTSmooth-v2.html) with `IgnoreXbins=True`, $n=20$ (where $1/n$ is the truncation for the Fourier transform), and smoothing order $order=2$.

In [ ]:
import mantid.simpleapi as mapi
import numpy as np
import scipp as sc

filename = 'PG3_4866_event.nxs'

In [ ]:
PG3_4866 = mapi.Load(Filename=filename)

binned = mapi.Rebin(InputWorkspace=PG3_4866, params='0.,8.33,16666.67')

In [ ]:
# extract 1 spectrum to plot
binned551 = mapi.ExtractSingleSpectrum(binned, 550)
ybin = binned551.readY(0)
errbin = binned551.readE(0)
x0 = binned551.readX(0)
# center the bins for the x-axis
xbin = 0.5*(x0[:-1] + x0[1:])

# create a dataset for Mantid's data to plot using scipp
dini = sc.Dataset(
   data = {'mantid': sc.array(dims=['tof'], 
                              values=ybin,
                              unit='counts')},
   coords = {'tof': sc.array(dims=['tof'],
                             values=xbin,
                             unit=sc.units.us)}
)

In [ ]:
sc.plot(dini,
        grid=True,
        errorbars=False,
        linestyle='solid',
        marker='',
        legend=True,
        title='Input spectrum of FFTSmooth in Mantid')

In [ ]:
binned_smooth = mapi.FFTSmooth(InputWorkspace=binned,
                               Filter='Butterworth',
                               Params='20,2',
                               IgnoreXBins=True,
                               AllSpectra=True)

In [ ]:
# Extract 1 spectrum to plot
binned_smooth551 = mapi.ExtractSingleSpectrum(binned_smooth, 550)

ysmth = binned_smooth551.readY(0)
x0 = binned_smooth551.readX(0)
xsmth = 0.5*(x0[:-1] + x0[1:])
esmth = binned_smooth551.readE(0)

dsmooth = sc.Dataset(
data={'mantid': sc.array(dims=['tof'], 
                         values=ysmth, 
                         unit='counts')},
coords={'tof':sc.array(dims=['tof'], 
                       values=xsmth, 
                       unit=sc.units.us)})

In [ ]:
sc.plot(dsmooth,
        grid=True,
        errorbars=False,
        linestyle='solid',
        marker='', 
        legend=True,
        title='Output spectrum of FFTSmooth in Mantid')

## scipp

In [ ]:
import scippneutron as scn
import fft_smooth

tof_min = 00000.00  # microseconds
tof_max = 16666.67  # microseconds

In [ ]:
vanadium = scn.load(filename=filename, 
                           load_pulse_times=True,
                           advanced_geometry=True,
                           mantid_args={'LoadMonitors': True})
vanadium

In [ ]:
tof_bins = sc.Variable(['tof'],
                       unit=sc.units.us,
                       values=np.linspace(tof_min, 
                                          tof_max, 
                                          num=2000))

In [ ]:
# check TOF step (this value is used for mantid binning parameters in the previous section)
(tof_max - tof_min) / 2000

In [ ]:
hist = sc.histogram(vanadium, tof_bins)
hist

In [ ]:
sc.plot(hist['spectrum', 550], 
        grid=True,
        title='Input spectrum of ffsmooth in scipp')

In [ ]:
vana_smooth = fft_smooth.fft_smooth(hist, 20, 2)
vana_smooth

In [ ]:
sc.plot(vana_smooth['spectrum', 550], grid=True, errorbars=False)

In [ ]:
# Extract 1 spectrum to plot and compare with Mantid
sc.plot({"Scipp": vana_smooth['spectrum', 550], 
         "Mantid": dsmooth['mantid']}, 
        errorbars=False, 
        linestyle='solid', 
        grid=True)

In [ ]:
# Display values of output spectrum from scipp and mantid
ysc = vana_smooth['spectrum', 550].values

for i in range(len(ysc)):
    print(ysmth[i] - ysc[i])